In [3]:
%matplotlib inline
import pandas as pd
from scipy.stats import pearsonr 
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

lang2mono = {'EN': 'bert-base-uncased',
             'ID': 'indobert-base-uncased',
             'FR': 'camembert-base',
             'TR': 'bert-base-turkish-uncased',
             'ZH': 'bert-base-chinese',
             'RU': 'rubert-base-cased',
             'DE': 'bert-base-german-dbmdz-uncased',
             'ES': 'bert-base-spanish-wwm-uncased'}

LANGS = ['EN', 'ID', 'FR', 'TR', 'ZH', 'RU', 'DE', 'ES']
model = 'PG'

In [6]:
def read_human_annotation(lang, types='focus'):
    if types=='focus':
        path_human = f'mturk/annotation_result/{lang}/human_focus_final.csv'
    else:
        assert types=='coverage'
        path_human = f'mturk/annotation_result/{lang}/human_coverage_final.csv'
    human = pd.read_csv(path_human)
    return human

def align(df, human_score, model):
    human = []
    machine = []
    #print(df)
    for idx, row in df.iterrows():
        human.append(human_score[(human_score['model'] == model) & (human_score['id']==row['fnames'])]['score'].values[0])
        machine.append(row['moverscore'])
    return human, machine

def read(lang, human_score, pretrained, model):
    path_BERT = f'mover_score/{lang}--BERT--{pretrained}.csv'
    path_PG = f'mover_score/{lang}--PG--{pretrained}.csv'
        
    humans = []; machines = []

    if model == 'BERT':
        df_BERT = pd.read_csv(path_BERT)
        human, machine = align(df_BERT, human_score, 'BERT')
        humans += human
        machines += machine
    elif model == 'PG':
        df_PG = pd.read_csv(path_PG)
        human, machine = align(df_PG, human_score, 'PG')
        humans += human
        machines += machine
    
    #print(humans)
    #print(machines)
    return pearsonr(humans, machines)[0], len(humans)

In [10]:
LANGS = ['EN', 'ID', 'FR', 'TR', 'ZH', 'RU', 'DE', 'ES']

# Focus
print('Focus', model)
print('=============')
cors = {}
for lang in LANGS:
    human_score = read_human_annotation(lang, 'focus')
    for pretrained in [ lang2mono[lang], 'bert-base-multilingual-cased', 'bert-base-multilingual-uncased', \
                      'xlm-roberta-base', 'xlm-roberta-large']:    
        cor, num = read(lang, human_score, pretrained, model)
        print(lang, num, pretrained, cor)    
    print()

Focus PG
EN 135 bert-base-uncased 0.6164471195287589
EN 135 bert-base-multilingual-cased 0.5740480716832725
EN 135 bert-base-multilingual-uncased 0.6335447636526799
EN 135 xlm-roberta-base 0.5572367589812162
EN 135 xlm-roberta-large 0.5259846122405507

ID 135 indobert-base-uncased 0.6685964663708452
ID 135 bert-base-multilingual-cased 0.6214476866881169
ID 135 bert-base-multilingual-uncased 0.65354736066805
ID 135 xlm-roberta-base 0.6103843658495229
ID 135 xlm-roberta-large 0.5744887968398797

FR 135 camembert-base 0.6341290414054612
FR 135 bert-base-multilingual-cased 0.7114173375145556
FR 135 bert-base-multilingual-uncased 0.757150751203846
FR 135 xlm-roberta-base 0.6001662754347832
FR 135 xlm-roberta-large 0.6038136261126427

TR 135 bert-base-turkish-uncased 0.8762992726882338
TR 135 bert-base-multilingual-cased 0.8425000351887733
TR 135 bert-base-multilingual-uncased 0.8754562378679195
TR 135 xlm-roberta-base 0.858844380394457
TR 135 xlm-roberta-large 0.8350149874305126

ZH 135 ber

In [11]:
# Coverage
print('Coverage', model)
print('=============')
cors = {}
for lang in LANGS:
    human_score = read_human_annotation(lang, 'coverage')
    for pretrained in [ lang2mono[lang], 'bert-base-multilingual-cased', 'bert-base-multilingual-uncased', \
                      'xlm-roberta-base', 'xlm-roberta-large']: 
        cor, num = read(lang, human_score, pretrained, model)
        print(lang, num, pretrained, cor)
    print()

Coverage PG
EN 135 bert-base-uncased 0.5744128253999503
EN 135 bert-base-multilingual-cased 0.531324766684457
EN 135 bert-base-multilingual-uncased 0.5789456830210102
EN 135 xlm-roberta-base 0.5638916955072275
EN 135 xlm-roberta-large 0.5227044211163876

ID 135 indobert-base-uncased 0.6487534517968864
ID 135 bert-base-multilingual-cased 0.6669593678347698
ID 135 bert-base-multilingual-uncased 0.6806911674137139
ID 135 xlm-roberta-base 0.60918956550856
ID 135 xlm-roberta-large 0.6225022599203025

FR 135 camembert-base 0.6258059520381745
FR 135 bert-base-multilingual-cased 0.6772067984519621
FR 135 bert-base-multilingual-uncased 0.739755021166638
FR 135 xlm-roberta-base 0.5195971110985669
FR 135 xlm-roberta-large 0.49871865824803285

TR 135 bert-base-turkish-uncased 0.7315959815042467
TR 135 bert-base-multilingual-cased 0.713899653879187
TR 135 bert-base-multilingual-uncased 0.7152166224245092
TR 135 xlm-roberta-base 0.6818916580799683
TR 135 xlm-roberta-large 0.6612647162648362

ZH 135 